In [7]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

In [9]:
# Scenario initialization
scenario = 'NetZ'  # alternative: 'BaU'

In [15]:
# Cooldown tracker
action_last_year = {}

# History containers
share_history     = {s: [] for s in generation_mix}
intensity_history = {s: [] for s in generation_intensities}
cost_history      = []
percent_lim_used_history = {s: [] for s in generation_mix}
abatement_records = []
yearly_flag_hist       = []
cumulative_flag_hist   = []
overspend_yearly       = []
overspend_cumulative   = []

In [17]:
def adjust_mix(old_mix, gen_source, delta):
    mix = old_mix.copy()
    primary = max(0.0, min(mix[gen_source] + delta, 1.0))
    mix[gen_source] = primary
    residual = 1.0 - primary
    others = [s for s in mix if s != gen_source]
    old_sum = sum(old_mix[s] for s in others)
    if old_sum > 0:
        for s in others:
            mix[s] = old_mix[s] / old_sum * residual
    elif others:
        for s in others:
            mix[s] = residual / len(others)
    total = sum(mix.values())
    if total <= 0:
        raise ValueError("Mix sums to zero")
    for s in mix:
        mix[s] /= total
    return mix

In [ ]:
def ABIM(current, target_int, mix, ints, opt, demand):
    action = opt['Action']
    src = opt['Source_key']
    ghg_eff = opt['GHGeffect_tonnesCO2perGWh']
    share_frac = opt['Shareeffect_GWhperyear'] / demand
    cost = opt['Costeffect_BUSD']

    # Capture starting intensity
    start_intensity = current

    # Simulate full application
    tmp_ints = ints.copy()
    tmp_ints[src] += ghg_eff
    tmp_mix = adjust_mix(mix, src, share_frac)
    grid_full = sum(tmp_mix[s] * tmp_ints[s] for s in tmp_mix)

    # Determine fraction needed to exactly hit the target (or full bite)
    if grid_full < target_int:
        frac = (current - target_int) / (current - grid_full)
    else:
        frac = 1.0

    # Apply GHG effect and share effect (clamped ≥ 0)
    ints[src] = max(ints[src] + ghg_eff * frac, 0.0)
    mix = adjust_mix(mix, src, share_frac * frac)
    new_grid = sum(mix[s] * ints[s] for s in mix)

    # Compute incremental cost
    cost_inc = cost * frac

    # Prepare action record
    action_record = {
        'Action': action,
        'Start Intensity': start_intensity,
        'End Intensity': new_grid,
        'Cumulative Cost': cost_inc
    }

    # Determine new current intensity (never below target)
    new_current = max(new_grid, target_int)

    return new_current, cost_inc, mix, ints, action_record

In [18]:
def apply_abatement_year(start_int, target_int, mix, ints, options, demand, year, last_years):
    total_cost = 0.0
    actions = []
    current = start_int

    while current > target_int:
        applied = False

        for _, opt in options.iterrows():
            action = opt['Action']
            cooldown = opt['Can_repeat_after_years']

            # Enforce repeat-after cooldown
            if year - last_years.get(action, -np.inf) < cooldown:
                continue

            # Call ABIM to implement this action
            new_current, cost_inc, mix, ints, record = ABIM(
                current, target_int, mix, ints, opt, demand
            )

            total_cost += cost_inc
            actions.append(record)

            # Update cooldown tracker and current intensity
            last_years[action] = year
            current = new_current
            applied = True

            # Stop early if target reached
            if current <= target_int:
                break

        if not applied:
            # No further actions can be applied
            break

    return current, total_cost, mix, ints, actions